In [1]:
import json
import pickle
import re

def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    print("Cleaned trailing spaces")

    return cleaned_data



def for_spacy(input_file):
    count = 0
    class_labels = list()
    training_data = []
    lines = []
    with open(input_file, 'r')  as f:
        lines = f.readlines()
    
    for line in lines:
        data = json.loads(line)
        entities = []
        if  data['annotation'] == None:
            continue
        text = data['content']
        count += 1
        for annotation in data['annotation']:
            point = annotation['points'][0]
            labels = annotation['label']
            
            for lb in labels:
                if lb not in class_labels:
                    class_labels.append(lb)
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                entities.append((point['start'], point['end'] + 1 ,label))


        training_data.append((text, {"entities" : entities}))
    training_data = trim_entity_spans(training_data)
    #print(training_data)
    print(count)
    
    output_file = '/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Spacy - Model/700/d.pkl'
    with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)
    class_labels = str(class_labels)
    output_file = '/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Spacy - Model/700/labels.txt'
    with open(output_file, 'w') as fp:
            fp.write(class_labels)
    
            
        
        
        


if __name__ == "__main__":
    for_spacy('/Users/cosmos/Documents/resume_parser_nlp-master/commons/Models/training/training_data/traindata.json')

Cleaned trailing spaces
693


In [2]:
import pickle
import json
import spacy
import random
import spacy
from spacy.util import minibatch, compounding 
from pathlib import Path
import os

#LABEL = ['Email Address', 'Links', 'Skills', 'Graduation Year', 'College Name', 'Degree', 'Companies worked at', 'Location', 'Name', 'Designation', 'projects', 'Years of Experience', 'Can Relocate to', 'UNKNOWN', 'Rewards and Achievements', 'Address', 'University', 'Relocate to', 'Certifications', 'state', 'links', 'College', 'training', 'des', 'abc']

def custom_nlp_train(filename):
    with open(filename, 'rb') as fp:
        doc = pickle.load(fp)
    
    ## Creating a blank spacy model
    nlp = spacy.blank('en')
    print("Created a blank en model")

    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
        print("Added ner pipe")
    else:
        ner.get_pipe('ner')

    # for i in LABEL:
    #     ner.add_label(i)


    # add labels
    for _, annotations in doc:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
        
    optimizer = nlp.begin_training()
    
    for itn in range(30):
                random.shuffle(doc)
                losses = {}
                # batch up the examples using spaCy's minibatch
                batches = minibatch(doc, size=compounding(4.0, 32.0, 1.001))
                for batch in batches:
                    texts, annotations = zip(*batch)
                    nlp.update(
                        texts,  # batch of texts
                        annotations,  # batch of annotations
                        drop=0.3,  # dropout - make it harder to memorise data
                        losses=losses,
                    )
                print("Losses", losses)

    output_dir = os.path.dirname(os.path.realpath(__file__)) + "/training_data"
    new_model_name="customspacy"
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()

    nlp.meta["name"] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)           
            

    


if __name__ == "__main__":
    custom_nlp_train('/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Spacy - Model/700/d.pkl')   

Init Plugin
Init Graph Optimizer
Init Kernel
Created a blank en model


ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.ner.EntityRecognizer object at 0x29960d2e0> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.